In [2]:
import numpy as np
import pandas as pd



import spacy
#from spacy.lemmatizer import Lemmatizer
from spacy.lang.en.stop_words import STOP_WORDS
import en_core_web_sm
from spacy.language import Language

from tqdm import tqdm_notebook as tqdm
from pprint import pprint

In [3]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

C:\Users\pgsin\anaconda3\envs\skippy\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [4]:
import os

print(os.getcwd())

c:\Users\pgsin\Documents\AI_ENGINEERING\Striving\Capstone\NLP


In [5]:
# # Function to Get the current 
# # working directory
# def current_path():
#     print("Current working directory before")
#     print(os.getcwd())
#     print()
   
   
# # Driver's code
# # Printing CWD before
# current_path()
   
# # Changing the CWD
# os.chdir('../')
   
# # Printing CWD after
# 
os.chdir('../')
print("Current working directory after")
print(os.getcwd())


Current working directory after
c:\Users\pgsin\Documents\AI_ENGINEERING\Striving\Capstone


In [6]:
nlp= spacy.load("en_core_web_sm")


In [7]:
for word in STOP_WORDS:
    lexeme = nlp.vocab[word]
    lexeme.is_stop = True

In [8]:
@Language.component("lemmy")
def lemmatizer(doc):
    # This takes in a doc of tokens from the NER and lemmatizes them. 
    # Pronouns (like "I" and "you" get lemmatized to '-PRON-', so I'm removing those.
    doc = [token.lemma_ for token in doc if token.lemma_ != '-PRON-']
    doc = u' '.join(doc)
    return nlp.make_doc(doc)
    
@Language.component("rstp")
def remove_stopwords(doc):
    # This will remove stopwords and punctuation.
    # Use token.text to return strings, which we'll need for Gensim.
    doc = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
    return doc


nlp.add_pipe("lemmy",after='ner')
nlp.add_pipe("rstp", last=True)

<function __main__.remove_stopwords(doc)>

In [9]:
with open("d2d_laptop.txt") as f:
    lines = f.readlines()
    #print(lines)
   

In [10]:
doc_list = []
# Iterates through each article in the corpus.
for doc in tqdm(lines):
    # Passes that article through the pipeline and adds to a new list.
    pr = nlp(doc)
    doc_list.append(pr)

C:\Users\pgsin\AppData\Local\Temp/ipykernel_20308/2979821374.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for doc in tqdm(lines):


  0%|          | 0/246 [00:00<?, ?it/s]

In [14]:
doc_list

[['right', 'laptop', '\n'],
 ['650', 'fantastic', '\n'],
 ['month', 'ago', 'announce', '\n'],
 ['laptop', 'game', '\n'],
 ['impression', 'like', 'okay', 'good', '\n'],
 ['stuff', '\n'],
 ['attempt', '\n'],
 ['stuff', 'right', 'laptop', '\n'],
 ['ve', 'expect', '\n'],
 ['good', 'great', 'right', '\n'],
 ['come', 'available', 'like', 'ta', '\n'],
 ['thing', '\n'],
 ['\n'],
 ['remarkably', 'good', 'money', '\n'],
 ['let', 'okay', 'thing', '\n'],
 ['initial', '\n'],
 ['reaction', 'like', 'run', '\n'],
 ['ad', 'right', '\n'],
 ['price', 'low', 've', 'run', '\n'],
 ['bloatware', 'crap', '\n'],
 ['people', 'want', 'people', 'need', '\n'],
 ['company', '\n'],
 ['laptop', 'company', '\n'],
 ['know', 'norton', 'mcafee', '\n'],
 ['crap', 'want', 'expect', '\n'],
 ['fill', '\n'],
 ['plain', 'blank', '\n'],
 ['slate', 'window', '10', 'like', '\n'],
 ['point', '\n'],
 ['idea', 'run', 'thing', '\n'],
 ['650', 'buck', 'let', 'talk', '\n'],
 ['build', 'thin', 'machine', 'aluminum', '\n'],
 ['device', '

In [11]:

# Creates, which is a mapping of word IDs to words.
words = corpora.Dictionary(doc_list)

# Turns each document into a bag of words.
corpus = [words.doc2bow(doc) for doc in doc_list]

In [12]:

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=words,
                                           num_topics=5, 
                                           random_state=2,
                                           update_every=1,
                                           passes=3,
                                           alpha='auto',
                                           per_word_topics=True)

In [13]:
pprint(lda_model.print_topics(num_words=10
))

[(0,
  '0.134*"\n'
  '" + 0.020*"open" + 0.019*"keyboard" + 0.019*"talk" + 0.019*"know" + '
  '0.018*"let" + 0.018*"good" + 0.017*"like" + 0.014*"hour" + '
  '0.014*"performance"'),
 (1,
  '0.085*"\n'
  '" + 0.029*"macbook" + 0.024*"leave" + 0.022*"like" + 0.021*"rough" + '
  '0.019*"edge" + 0.018*"clone" + 0.013*"replicate" + 0.013*"sell" + '
  '0.013*"let"'),
 (2,
  '0.120*"\n'
  '" + 0.036*"macbook" + 0.029*"keyboard" + 0.025*"ve" + 0.023*"look" + '
  '0.022*"people" + 0.017*"thing" + 0.016*"like" + 0.015*"type" + '
  '0.009*"high"'),
 (3,
  '0.263*"\n'
  '" + 0.060*"like" + 0.023*"right" + 0.023*"thing" + 0.017*"kind" + '
  '0.015*"screen" + 0.013*"good" + 0.012*"company" + 0.010*"device" + '
  '0.010*"laptop"'),
 (4,
  '0.182*"\n'
  '" + 0.019*"device" + 0.018*"usb" + 0.018*"kind" + 0.017*"point" + '
  '0.015*"like" + 0.015*"cut" + 0.015*"macbook" + 0.014*"little" + 0.014*"c"')]
